In [1]:
from pathlib import Path
from fewshot.feature_extractors import load_feature_extractor, FEATURE_EXTRACTORS
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
import pandas as pd

DATA_DIR = Path("/app/data/CRC-100K-VIT/")

# Set feature extractor here!
FEATURE_EXTRACTOR = "phikon"
# FEATURE_EXTRACTOR = "vit"
FEATURE_EXTRACTOR = "resnet50"

print(f"Available feature extractors: {', '.join(FEATURE_EXTRACTORS.keys())}")
print(f"Using feature extractor: {FEATURE_EXTRACTOR}")

In [2]:
feature_extractor = load_feature_extractor(FEATURE_EXTRACTOR)

images = list(DATA_DIR.glob("*.png"))

In [18]:
to_tensor = transforms.ToTensor()

# Extract features
features = {
    image.name: feature_extractor(to_tensor(Image.open(image)).unsqueeze(0)).squeeze(0)
    for image in tqdm(images, desc="Extracting features")
}

Extracting features: 100%|██████████| 135/135 [00:18<00:00,  7.18it/s]


In [19]:
binary_dir = DATA_DIR / "binary"
predictions_dir = DATA_DIR / "predictions"
predictions_dir.mkdir(exist_ok=True)

gpt_results_files = list(binary_dir.glob("**/*.csv"))

print(f"Accuracy on GPT results files using nearest neighbor (L2 distance) on features extracted by {feature_extractor.name}:")

for gpt_results_file in gpt_results_files:
    if "zero_shot" in gpt_results_file.name:
        continue
    gpt_results = pd.read_csv(gpt_results_file, index_col=0, header=0)
    gpt_results["train_data"] = gpt_results["train_data"].map(eval)

    for i, experiment in gpt_results.iterrows():
        test_label = experiment["label"]
        test_image = experiment["fname"] + ".png"
        train_images = experiment["train_data"]

        # Get features of training and test images
        train_features = {
            features[image]: label
            for image, label in train_images.items()
        }
        test_feature = features[test_image]

        # Compute distance to each training feature
        dists = [
            ((feature - test_feature).norm().item(), label)
            for feature, label in train_features.items()
        ]

        # Get label of closest training feature
        _, pred_label = min(dists)

        gpt_results.loc[i, "pred_label"] = pred_label
    
    output_file = predictions_dir / gpt_results_file.relative_to(binary_dir)
    output_file.parent.mkdir(exist_ok=True, parents=True)
    gpt_results.to_csv(output_file)

    accuracy = (gpt_results["label"] == gpt_results["pred_label"]).mean()
    print(f"{gpt_results_file.name}: {accuracy:.2f}")

Accuracy on GPT results files using nearest neighbor (L2 distance) on features extracted by resnet50:
results_ten_shot_run_high_interleaved_run2.csv: 0.87
results_ten_shot_run_high_interleaved_run1.csv: 0.77
results_five_shot_run_high_interleaved_run1.csv: 0.77
results_five_shot_run_high_interleaved_run2.csv: 0.73
results_three_shot_run_high_interleaved_run1.csv: 0.70
results_three_shot_run_high_interleaved_run2.csv: 0.60
